In [1]:
# 1 Paso - Instalar PySpark en el Contexto de Colab (si es que no está instalado)
!pip install -q PySpark

In [3]:
# 2 Paso - Conectarme y crear la sesión en Apache Spark (SparkSession)
from pyspark.sql import SparkSession
# Crear la sesión
spark = SparkSession.builder.appName("ProyectoMetroSantiago").config("spark.executor.memory","1g").config("spark.driver.memory","1g").config("spark.sql.shuffle.partitions","2").getOrCreate()

In [5]:
# Cargar el CSV en un DF (asegurarse de subir antes el archivo a la carpeta interna de recursos)
df = spark.read.csv("metro_santiago.csv",header=True,inferSchema=True)

#Explrar este DF (Apache Spark) con show
df.show()

+---------+-----+--------------+--------------------+----------------+-------------------+
|id_evento|línea|      estación|      tipo_incidente|duración_minutos|               hora|
+---------+-----+--------------+--------------------+----------------+-------------------+
|        1|   L2|      Lo Prado|     Falla eléctrica|              15|2025-08-23 11:45:00|
|        2|   L5|   Irarrázaval|          Evacuación|              15|2025-08-23 20:00:00|
|        3|   L4|     San Pablo|     Falla eléctrica|              25|2025-08-23 19:00:00|
|        4|   L1|      Lo Prado|     Falla eléctrica|              30|2025-08-23 05:45:00|
|        5|   L3|   Irarrázaval|     Falla eléctrica|              20|2025-08-23 08:00:00|
|        6|   L2|    Los Leones|       Mantenimiento|              30|2025-08-23 17:00:00|
|        7|   L3|         Ñuñoa|       Mantenimiento|               5|2025-08-23 12:30:00|
|        8|   L1|    Los Héroes|     Falla eléctrica|              30|2025-08-23 08:30:00|

In [7]:
# Transformar el df a RDD
rdd = df.rdd

# Mostrar algunos registros del RDD
rdd.take(5)

[Row(id_evento=1, línea='L2', estación='Lo Prado', tipo_incidente='Falla eléctrica', duración_minutos=15, hora=datetime.datetime(2025, 8, 23, 11, 45)),
 Row(id_evento=2, línea='L5', estación='Irarrázaval', tipo_incidente='Evacuación', duración_minutos=15, hora=datetime.datetime(2025, 8, 23, 20, 0)),
 Row(id_evento=3, línea='L4', estación='San Pablo', tipo_incidente='Falla eléctrica', duración_minutos=25, hora=datetime.datetime(2025, 8, 23, 19, 0)),
 Row(id_evento=4, línea='L1', estación='Lo Prado', tipo_incidente='Falla eléctrica', duración_minutos=30, hora=datetime.datetime(2025, 8, 23, 5, 45)),
 Row(id_evento=5, línea='L3', estación='Irarrázaval', tipo_incidente='Falla eléctrica', duración_minutos=20, hora=datetime.datetime(2025, 8, 23, 8, 0))]

In [8]:
# Almacenar "in .memory" (en memoria cache) nuestro RDD para utilizarlo varias veces
rdd.cache()

MapPartitionsRDD[28] at javaToPython at NativeMethodAccessorImpl.java:0

In [9]:
# Ver cuantas particiones tiene nuestro RDD
rdd.getNumPartitions()

1

In [10]:
# Reparticionar nuestro RDD en 4 particiones
rdd_repartido = rdd.repartition(4)

rdd_repartido.getNumPartitions()

4

In [11]:
# Transformaciones - Son operaciones a realizar en el RDD que lo afectan, cerando un nuevo objeto RDD
rdd_filtrado = rdd.filter(lambda row: int(row['duración_minutos']) > 20)

rdd_filtrado.take(5)

[Row(id_evento=3, línea='L4', estación='San Pablo', tipo_incidente='Falla eléctrica', duración_minutos=25, hora=datetime.datetime(2025, 8, 23, 19, 0)),
 Row(id_evento=4, línea='L1', estación='Lo Prado', tipo_incidente='Falla eléctrica', duración_minutos=30, hora=datetime.datetime(2025, 8, 23, 5, 45)),
 Row(id_evento=6, línea='L2', estación='Los Leones', tipo_incidente='Mantenimiento', duración_minutos=30, hora=datetime.datetime(2025, 8, 23, 17, 0)),
 Row(id_evento=8, línea='L1', estación='Los Héroes', tipo_incidente='Falla eléctrica', duración_minutos=30, hora=datetime.datetime(2025, 8, 23, 8, 30)),
 Row(id_evento=9, línea='L3', estación='Tobalaba', tipo_incidente='Evacuación', duración_minutos=25, hora=datetime.datetime(2025, 8, 23, 16, 15))]

In [12]:
# TRANSFORMACIÓN - FILTER: aplica un filtro o modifica el RDD en base a un criterio lógico
rdd_evacuaciones = rdd.filter(lambda row: row['tipo_incidente'] == 'Evacuación')
rdd_evacuaciones.take(5)

[Row(id_evento=2, línea='L5', estación='Irarrázaval', tipo_incidente='Evacuación', duración_minutos=15, hora=datetime.datetime(2025, 8, 23, 20, 0)),
 Row(id_evento=9, línea='L3', estación='Tobalaba', tipo_incidente='Evacuación', duración_minutos=25, hora=datetime.datetime(2025, 8, 23, 16, 15)),
 Row(id_evento=11, línea='L2', estación='Plaza Egaña', tipo_incidente='Evacuación', duración_minutos=10, hora=datetime.datetime(2025, 8, 23, 12, 15)),
 Row(id_evento=15, línea='L4', estación='Los Héroes', tipo_incidente='Evacuación', duración_minutos=30, hora=datetime.datetime(2025, 8, 23, 15, 15)),
 Row(id_evento=18, línea='L3', estación='Irarrázaval', tipo_incidente='Evacuación', duración_minutos=10, hora=datetime.datetime(2025, 8, 23, 5, 30))]

In [13]:
# TRANSFORMACIÓN - MAP: Función que aplica a cada elemento del RDD y devuelve uno nuevo transformado
rdd_estaciones_mayusculas = rdd.map(lambda row: row['estación'].upper())
rdd_estaciones_mayusculas.take(5)

['LO PRADO', 'IRARRÁZAVAL', 'SAN PABLO', 'LO PRADO', 'IRARRÁZAVAL']

In [15]:
# TRANSFORMACIÓN - FLATMAP: Función que, como map(),permite devolver elementos transformados pero varios de una sola entrada
rdd_estaciones = rdd.map(lambda row: row['estación'])
palabras = rdd_estaciones.flatMap(lambda x: x.split(" "))
palabras.take(5)

['Lo', 'Prado', 'Irarrázaval', 'San', 'Pablo']

In [20]:
# TRANSFORMACIÓN - SAMPLE: Permite extraer una muestra aleatoria del RDD
rdd_muestra = rdd.sample(seed=42,fraction=0.3,withReplacement=False)
rdd_muestra.take(5)

[Row(id_evento=10, línea='L4A', estación='Los Dominicos', tipo_incidente='Interrupción de servicio', duración_minutos=30, hora=datetime.datetime(2025, 8, 23, 16, 45)),
 Row(id_evento=11, línea='L2', estación='Plaza Egaña', tipo_incidente='Evacuación', duración_minutos=10, hora=datetime.datetime(2025, 8, 23, 12, 15)),
 Row(id_evento=14, línea='L5', estación='Pajaritos', tipo_incidente='Mantenimiento', duración_minutos=15, hora=datetime.datetime(2025, 8, 23, 13, 30)),
 Row(id_evento=17, línea='L1', estación='Vicente Valdés', tipo_incidente='Falla eléctrica', duración_minutos=25, hora=datetime.datetime(2025, 8, 23, 20, 15)),
 Row(id_evento=18, línea='L3', estación='Irarrázaval', tipo_incidente='Evacuación', duración_minutos=10, hora=datetime.datetime(2025, 8, 23, 5, 30))]

In [21]:
# TRANSFORMACIÓN - UNION: Une 2 RDDs (sin eliminar duplicados)
rdd_duplicado = rdd.filter(lambda row: row['tipo_incidente'] == "Falla eléctrica")
rdd_unido = rdd_evacuaciones.union(rdd_duplicado)
rdd_unido.take(5)

[Row(id_evento=2, línea='L5', estación='Irarrázaval', tipo_incidente='Evacuación', duración_minutos=15, hora=datetime.datetime(2025, 8, 23, 20, 0)),
 Row(id_evento=9, línea='L3', estación='Tobalaba', tipo_incidente='Evacuación', duración_minutos=25, hora=datetime.datetime(2025, 8, 23, 16, 15)),
 Row(id_evento=11, línea='L2', estación='Plaza Egaña', tipo_incidente='Evacuación', duración_minutos=10, hora=datetime.datetime(2025, 8, 23, 12, 15)),
 Row(id_evento=15, línea='L4', estación='Los Héroes', tipo_incidente='Evacuación', duración_minutos=30, hora=datetime.datetime(2025, 8, 23, 15, 15)),
 Row(id_evento=18, línea='L3', estación='Irarrázaval', tipo_incidente='Evacuación', duración_minutos=10, hora=datetime.datetime(2025, 8, 23, 5, 30))]

In [22]:
# TRANSFORMACIÓN - DISTINCT: Elimina los duplicados (igual que SQL)
rdd_estaciones_distintas = rdd.map(lambda row: row['estación']).distinct()
rdd_estaciones_distintas.take(5)

['Lo Prado', 'Irarrázaval', 'San Pablo', 'Los Leones', 'Ñuñoa']

In [23]:
# TRANSFORMACIÓN - SORTBY: Ordena los elementos segun criterio
rdd_ordenado = rdd.sortBy(lambda row: int(row['duración_minutos']))
rdd_ordenado.take(5)

[Row(id_evento=7, línea='L3', estación='Ñuñoa', tipo_incidente='Mantenimiento', duración_minutos=5, hora=datetime.datetime(2025, 8, 23, 12, 30)),
 Row(id_evento=13, línea='L5', estación='Los Héroes', tipo_incidente='Falla eléctrica', duración_minutos=5, hora=datetime.datetime(2025, 8, 23, 17, 0)),
 Row(id_evento=24, línea='L5', estación='Pajaritos', tipo_incidente='Mantenimiento', duración_minutos=5, hora=datetime.datetime(2025, 8, 23, 9, 15)),
 Row(id_evento=25, línea='L2', estación='Tobalaba', tipo_incidente='Falla eléctrica', duración_minutos=5, hora=datetime.datetime(2025, 8, 23, 23, 15)),
 Row(id_evento=40, línea='L5', estación='Los Dominicos', tipo_incidente='Evacuación', duración_minutos=5, hora=datetime.datetime(2025, 8, 23, 18, 45))]

In [ ]:
# ACCIONES - Métodos que no afectan al RDD, solo despliegan determinada info.
# Al ejecutarse, también ejecutan todas las transformaciones programadas

#take(n) -Toma y despliega un listado de los primeros elementos (n)
# collect() - Toma y despliega toda la información del RDD (no utilizar con RDDs muy poblados)
# top(n) - Toma y muestra los primeros 'n' datos ordenados en su orden natural, solo aplicado para columnas numéricas
# sum() - Despliega la suma de elementos, para columnas numéricas
# mean() - Despliega el promedio de los elementos, para columnas numéricas
# stdev() - Despliega la desv. Est. (dispersión), para columnas numéricas

# Ejercicio de la clase

In [42]:
spark = SparkSession.builder.appName("ProyectoHospitalSantiago").config("spark.executor.memory","1g").config("spark.driver.memory","1g").config("spark.sql.shuffle.partitions","2").getOrCreate()

In [43]:
sc=spark.sparkContext

In [44]:
rdd=sc.textFile("ejercicio_hospital_santiago.csv")

In [45]:
rdd = df.rdd
rdd.take(5)

[Row(paciente_id=1, nombre='María', apellido='González', edad=45, sexo='F', comuna='Santiago', servicio_clinico='Medicina Interna', fecha_ingreso=datetime.date(2024, 1, 15), diagnostico_principal='Hipertensión arterial', dias_hospitalizacion=3, costo_total=450000, medico_tratante='Dr. Rodríguez', estado_alta='Alta médica'),
 Row(paciente_id=2, nombre='Juan', apellido='Pérez', edad=67, sexo='M', comuna='Providencia', servicio_clinico='Cardiología', fecha_ingreso=datetime.date(2024, 1, 18), diagnostico_principal='Infarto agudo miocardio', dias_hospitalizacion=7, costo_total=1200000, medico_tratante='Dr. Silva', estado_alta='Alta médica'),
 Row(paciente_id=3, nombre='Carmen', apellido='López', edad=34, sexo='F', comuna='Las Condes', servicio_clinico='Ginecología', fecha_ingreso=datetime.date(2024, 1, 20), diagnostico_principal='Parto normal', dias_hospitalizacion=2, costo_total=350000, medico_tratante='Dra. Morales', estado_alta='Alta médica'),
 Row(paciente_id=4, nombre='Pedro', apellido

In [46]:
# TRANSFORMACIÓN - FLATMAP: Función que, como map(),permite devolver elementos transformados pero varios de una sola entrada
rdd_medicos = rdd.map(lambda row: row["medico_tratante"])
palabras = rdd_medicos.flatMap(lambda x: x.split(" "))
palabras.take(5)

['Dr.', 'Rodríguez', 'Dr.', 'Silva', 'Dra.']

In [48]:
rdd_filtrado = rdd.filter(lambda row: len(row["nombre"]) >= 4 or len(row["comuna"]) >= 4)
rdd_filtrado.take(5)

[Row(paciente_id=1, nombre='María', apellido='González', edad=45, sexo='F', comuna='Santiago', servicio_clinico='Medicina Interna', fecha_ingreso=datetime.date(2024, 1, 15), diagnostico_principal='Hipertensión arterial', dias_hospitalizacion=3, costo_total=450000, medico_tratante='Dr. Rodríguez', estado_alta='Alta médica'),
 Row(paciente_id=2, nombre='Juan', apellido='Pérez', edad=67, sexo='M', comuna='Providencia', servicio_clinico='Cardiología', fecha_ingreso=datetime.date(2024, 1, 18), diagnostico_principal='Infarto agudo miocardio', dias_hospitalizacion=7, costo_total=1200000, medico_tratante='Dr. Silva', estado_alta='Alta médica'),
 Row(paciente_id=3, nombre='Carmen', apellido='López', edad=34, sexo='F', comuna='Las Condes', servicio_clinico='Ginecología', fecha_ingreso=datetime.date(2024, 1, 20), diagnostico_principal='Parto normal', dias_hospitalizacion=2, costo_total=350000, medico_tratante='Dra. Morales', estado_alta='Alta médica'),
 Row(paciente_id=4, nombre='Pedro', apellido

In [50]:
# TRANSFORMACIÓN - MAP: Función que aplica a cada elemento del RDD y devuelve uno nuevo transformado
rdd_par = rdd.map(lambda row: (row['nombre'],1))
rdd_par.take(5)

[('María', 1), ('Juan', 1), ('Carmen', 1), ('Pedro', 1), ('Ana', 1)]

In [53]:
rdd_frecuencia = rdd_par.reduceByKey(lambda a,b:a+b)
for element in rdd_frecuencia.collect():
  print(element)

('María', 1)
('Juan', 1)
('Carmen', 1)
('Pedro', 1)
('Ana', 1)
('Luis', 1)
('Rosa', 1)
('Carlos', 1)
('Elena', 1)
('Jorge', 2)
('Patricia', 1)
('Roberto', 1)
('Claudia', 1)
('Miguel', 1)
('Gloria', 1)
('Fernando', 2)
('Mónica', 2)
('Andrés', 1)
('Beatriz', 1)
('Daniel', 1)
('Soledad', 1)
('Raúl', 1)
('Francisca', 2)
('Rodrigo', 2)
('Verónica', 1)
('Patricio', 1)
('Isabel', 1)
('Gonzalo', 1)
('Lorena', 1)
('Sergio', 1)
('Alejandra', 2)
('Marcelo', 1)
('Carolina', 2)
('Hugo', 1)
('Pamela', 1)
('Cristián', 1)
('Mariana', 1)
('Javier', 1)
('Paulina', 1)
('Eduardo', 1)
('Antonio', 1)
('Carla', 1)
('Manuel', 1)
('Viviana', 1)
('Ricardo', 1)
('Eugenia', 1)
('Álvaro', 1)
('Nadia', 1)
('Óscar', 1)
('Ingrid', 1)
('Tomás', 1)
('Luz', 1)
('Pablo', 1)
('Ximena', 1)
('Ignacio', 1)
('Andrea', 1)
('Guillermo', 1)
('Pilar', 1)
('Esteban', 1)
('Macarena', 1)
('Nelson', 1)
('Constanza', 1)
('Daniela', 1)
('Mauricio', 1)
('Silvia', 1)
('Claudio', 1)
('Gabriela', 1)
('Hernán', 1)
('Loreto', 1)
('Iván', 1)


In [55]:
# TRANSFORMACIÓN - SORTBY: Ordena los elementos segun criterio
rdd_ordenado = rdd_frecuencia.sortBy(lambda a,b: b, ascending=False)
rdd_ordenado.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 43.0 failed 1 times, most recent failure: Lost task 0.0 in stage 43.0 (TID 40) (f19e41e422f5 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/lib/python3.12/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pyspark/rdd.py", line 5434, in pipeline_func
   